HW 2: I worked with Emma Brown and Jocelyn Rosa to complete this assignment!

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import dhs_util
from dhs_util import *

In [ ]:
os.chdir('/Users/katielark/Downloads/data_viz')
df = pd.read_csv('dhs_service_records_synthesized_final.csv')
df = dhs_preprocessing(df)
df, service_map = add_service_label(df)
df = add_age_bin(df)

recipient = get_recipient_attribute(df)

In [ ]:
df

In [ ]:
recipient

1. Choose a pair or multiple behavior cohorts of recipients based on a set of behavior criteria that you think may be of importance (to the works done at DHS)
- e.g., recipients who received mental health service (S14) vs. those who did not
- e.g., recipients who received mental health service AND income support vs. those who received mental health support only (or the other way around)

In [ ]:
s15_id =df.query("serv == 'S15'")['id'].unique()
len(s15_id)

In [ ]:
one_serv_id = recipient.query("distinct_service == 1").id.to_list()
len(one_serv_id)

In [ ]:
s15_only = list(set(s15_id) & set(one_serv_id))
len(s15_only)

In [ ]:
s14_id =df.query("serv == 'S14'")['id'].unique()
len(s14_id)

In [ ]:
s14_s15 = list(set(s14_id) & set(s15_id))
len(s14_s15)

In [ ]:
s_15 = list(set(s15_id)-set(s14_s15)-set(s15_only))
len(s_15)

In [ ]:
cohort1 = pd.DataFrame(zip(s14_s15, ['ch1']*len(s14_s15)),columns=['id','cohort']) #cohort for ids receiving BOTH substance abuse AND mental health services
cohort2 = pd.DataFrame(zip(s_15, ['ch2']*len(s_15)),columns=['id','cohort']) #cohort for ids receiving mental health services (excluding mental health and only 1 service)
cohort3 = pd.DataFrame(zip(s15_only, ['ch3']*len(s15_only)),columns=['id','cohort']) #cohort for ids receiving ONLY substance abuse services
cohort_all=pd.concat([cohort1,cohort2,cohort3],axis=0) 

In [ ]:
len(cohort_all) #verifying!

In [ ]:
cohort_all.groupby('cohort').id.nunique() #verifying!

2. Prepare the analysis file for cohort analysis on your cohorts

I am creating a cleaned dataframe that I will use in the analysis that follows! 

In [ ]:
df = df.merge(cohort_all, on=['id'], how='inner') #merging df w cohort to make df w all ids in every cohort

In [ ]:
df

In [ ]:
df.groupby('cohort').nunique() #verifying!

3. Produce an analysis to answer questions that you think may be insightful
- e.g., what are the demographic differences between the cohorts?
- e.g., are there common services (other than income support) that the cohorts have in common?
- What are some similarities between the cohorts? 

1. What are the differences in race between the cohorts?

In [ ]:
df1 = df.groupby(['race','cohort']).agg(
    num_recipient= ('id', 'nunique')
).reset_index()

In [ ]:
df1

In [ ]:
groups = df1.groupby("cohort")
fig, axes = plt.subplots(1, len(groups), sharey=True, figsize=(14,8))

for ax, (cohort, group) in zip(axes, groups):
    # The rename_axis function makes the difference
    group.set_index("race").rename_axis(cohort)["num_recipient"].plot(kind="bar", ax=ax)
    ax.tick_params(axis='both', which='both', length=0)

ax.legend()
fig.subplots_adjust(wspace=0)

Some observations from the graphs/dataframe created above: 

More white people receive S15 and S14/S15 than any other race. This could be due to multiple reasons, and likely these reasons confound each other. There is a higher proportion of white people living in Allegheny county, and also there is a higher proportion of white people receiving DHS services in total in Allegheny county. A proportional analysis of S15 and the demographics of Allegheny county would be interesting to investigate. 

There is an almost even number of black recipients in Cohorts 1 and Cohorts 2, whereas there is a higher number of white recipients in Cohort 2 than Cohort 1. This indicates that a higher percentage of white recipients are receiving both mental health services in addition to substance abuse services than black recipients. This pattern, though subtle and less significant, is also seen in the Asian recipients as well. 

2. What are the differences in gender between the cohorts?

In [ ]:
df2 = df.groupby(['gender','cohort']).agg(
    num_recipient= ('id', 'nunique')
).reset_index()

In [ ]:
groups = df2.groupby("cohort")
fig, axes = plt.subplots(1, len(groups), sharey=True, figsize=(14,8))

for ax, (cohort, group) in zip(axes, groups):
    # The rename_axis function makes the difference
    group.set_index("gender").rename_axis(cohort)["num_recipient"].plot(kind="bar", ax=ax)
    ax.tick_params(axis='both', which='both', length=0)

ax.legend()
fig.subplots_adjust(wspace=0)

Some observations from the grpahs above:

There is very little significant difference between men and women within the two cohorts. 

3. Did the 38 recipients who only used s15 receive care multiple months in a year?

In [ ]:
df38 = df[df['cohort'] == 'ch3']
df38

In [ ]:
df38_g = df38.groupby(['id']).agg(
    num_months= ('month', 'nunique')
).reset_index()

In [ ]:
df38_g

In [ ]:
df38_g['num_months'].mean()

I am very interested by this cohort. In my knowledge of substance abuse, it is often accompanied by a variety of other conditions that demand different services. I am glad to see that there are only 38 recipients in this data that have only sought out substance abuse services, as that implies that most other people seeking this service are being assisted in other ways. I predicted that a majority of these recipients would only have recieved this service once because if they kept returning, DHS would likely encourage them to seek other services as well. And there are many S15 recipients who only received the service once. However, there are a much higher number of recipients who received it 10-12 times than I anticipated. 

4. What are the common services that the recipients in cohorts 1 and 2 have in common?

In [ ]:
df1415 = df[df['cohort'] == 'ch1']

In [ ]:
df1415_g = df1415.groupby(['serv']).agg(
    num_recipients= ('id', 'nunique')
).reset_index()

In [ ]:
df1415_g

In [ ]:
df15 = df[df['cohort'] == 'ch2']

In [ ]:
df15_g = df15.groupby(['serv']).agg(
    num_recipients= ('id', 'nunique')
).reset_index()

In [ ]:
df15_g

In [ ]:
sns.lineplot(data=df1415_g, x='serv', y='num_recipients', label='df1415')
sns.lineplot(data=df15_g, x='serv', y='num_recipients', label='df15')
plt.xticks(rotation=90)
plt.legend()
plt.show()
#I used the code from workbook 4 to build this graph, but I used chatgpt to learn how to plot two lines associated with two dataframes on the same plot.

Alright, some analyses from the graph above:

Aside from the obvious things (df15 having  no S14 and both df1415 and df15 having lots of S15), it is interesting to see that both dataframes show that many recipients also receive S12 (income support- the most common service) and S09 (DHS services- I am definintely confused by that column... aren't these all DHS services?) 

Additionally, S17 (Jail population) and S21 (Parents receiving child welfare) make notable appearance. 

Notably, neither category has high occurrences of "Overdoses", which makes me think that those are marked entirely separate from substance abuse... and there are fewer of them altogether. 

Neither service has high instances of the services relating to children. 

What is most important to see from this graph, however, is that most people who seek out services for substance abuse do so in conjunction with other services. 